In [ ]:
import osmnx
import pandas as pd

This notebook takes vehicle trajectories and routes them on the network finding the shortest path.

In [2]:
if "snakemake" in locals():
    trajectories_path = snakemake.input["trajectories"]
    nodes_path = snakemake.input["nodes"]
    network_path = snakemake.input["network"]
    output_path = snakemake.output[0]
else:
    trajectories_path = "../../resources/operators/synthetic1/trajectories.csv"
    nodes_path = "../../results/operators/synthetic1/nodes.parquet"
    network_path = "../../results/network/network.graphml"
    output_path = "../../results/operators/synthetic1/routes.parquet"

In [3]:
# Load graph
osmnx.io._convert_bool_string = lambda x: bool(x) if not x is None else False
graph = osmnx.load_graphml(network_path)

In [4]:
# Load trajectories
df_trajectories = pd.read_csv(trajectories_path)[["vehicle_id", "date"]]

# Load nodes
df_nodes = pd.read_parquet(nodes_path)

# Merge
df_trajectories["node"] = df_nodes["node"].values

In [5]:
# Convert stop sequences into route segments per row
df_routes = []

for index, df_group in df_trajectories.groupby(["vehicle_id", "date"]):
    vehicle_id, date = index

    df_routes.append(pd.DataFrame({
        "vehicle_id": vehicle_id, "date": date,
        "origin": df_group["node"].values[:-1],
        "destination": df_group["node"].values[1:],
    }))

df_routes = pd.concat(df_routes)

In [7]:
# Route unique relations
df_relations = df_routes[["origin", "destination"]].drop_duplicates()

df_relations["route"] = osmnx.shortest_path(graph, 
    df_relations["origin"], df_relations["destination"], weight = "travel_time")

In [8]:
# Merge back onto the routes
df_routes = pd.merge(df_routes, df_relations, on = ["origin", "destination"])

In [9]:
# Output
df_routes.to_parquet(output_path)